In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'drive/My Drive/Final_year_project/Final_Project_Notes/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/Final_year_project/Final_Project_Notes/resnet_utils.py" "resnet_utils.py"

In [0]:
#%pip install tensorflow==1.13.1
!pip install tensorflow_gpu

     |████████████████████████████████| 421.8MB 28kB/s 
     |████████████████████████████████| 450kB 36.6MB/s 
     |████████████████████████████████| 3.9MB 40.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=658c53372f83549f8ad975e2315b86083873b03de95f06b42f057598e9fa7cdd
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnet_utils import *
import glob
import os
import shutil 
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img


Using TensorFlow backend.


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255)

In [0]:
train_dataset = train_datagen.flow_from_directory('/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Split/train',
                                                 target_size=(256, 256), batch_size=64, class_mode='categorical', shuffle=True)
valid_dataset = test_datagen.flow_from_directory('/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Split/val',
                                                 target_size=(256, 256), batch_size=64, class_mode='categorical', shuffle=True)

Found 56468 images belonging to 19 classes.
Found 15528 images belonging to 19 classes.


In [0]:
print(train_dataset.class_indices)
print(valid_dataset.class_indices)

{'Abstract_Expressionism': 0, 'Art_Nouveau_Modern': 1, 'Baroque': 2, 'Color_Field_Painting': 3, 'Cubism': 4, 'Early_Renaissance': 5, 'Expressionism': 6, 'High_Renaissance': 7, 'Impressionism': 8, 'Mannerism_Late_Renaissance': 9, 'Minimalism': 10, 'Naive_Art_Primitivism': 11, 'Northern_Renaissance': 12, 'Pop_Art': 13, 'Post_Impressionism': 14, 'Realism': 15, 'Rococo': 16, 'Romanticism': 17, 'Symbolism': 18}
{'Abstract_Expressionism': 0, 'Art_Nouveau_Modern': 1, 'Baroque': 2, 'Color_Field_Painting': 3, 'Cubism': 4, 'Early_Renaissance': 5, 'Expressionism': 6, 'High_Renaissance': 7, 'Impressionism': 8, 'Mannerism_Late_Renaissance': 9, 'Minimalism': 10, 'Naive_Art_Primitivism': 11, 'Northern_Renaissance': 12, 'Pop_Art': 13, 'Post_Impressionism': 14, 'Realism': 15, 'Rococo': 16, 'Romanticism': 17, 'Symbolism': 18}


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5472433259369197886, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14541318844196768993
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17189197890848178129
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2812915418376753643
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
!pip install resnet


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
img_height,img_width = 256, 256
num_classes = 19
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
resnet = ResNet50(weights= "imagenet", include_top=False, input_shape= (img_height,img_width,3))
output = resnet.layers[-1].output
output = keras.layers.Flatten()(output)
resnet = Model(resnet.input, output=output)
for layer in resnet.layers:
    layer.trainable = False
resnet.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("fl...)`
  # This is added back by InteractiveShellApp.init_path()


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(resnet)
model.add(Dense(512, activation='relu', input_dim=(img_height,img_width,3)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(27, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 131072)            23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               67109376  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 27)                13851     
Total params: 90,973,595
Trainable params: 67,385,883
Non-trainable params: 23,587,712
_________________________________

In [0]:
resnet.trainable = True
set_trainable = False
for layer in resnet.layers:
    if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
layers = [(layer, layer.name, layer.trainable) for layer in resnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_1,False
1,<keras.layers.convolutional.ZeroPadding2D obje...,conv1_pad,False
2,<keras.layers.convolutional.Conv2D object at 0...,conv1,False
3,<keras.layers.normalization.BatchNormalization...,bn_conv1,False
4,<keras.layers.core.Activation object at 0x7f8b...,activation_1,False
...,...,...,...
171,<keras.layers.convolutional.Conv2D object at 0...,res5c_branch2c,True
172,<keras.layers.normalization.BatchNormalization...,bn5c_branch2c,True
173,<keras.layers.merge.Add object at 0x7f8ad20bdc18>,add_16,True
174,<keras.layers.core.Activation object at 0x7f8a...,activation_49,True


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model_finetuned = Sequential()
model_finetuned.add(resnet)
model_finetuned.add(Dense(512, activation='relu', input_dim=(img_height,img_width,3)))
model_finetuned.add(Dropout(0.3))
model_finetuned.add(Dense(512, activation='relu'))
model_finetuned.add(Dropout(-1.0))
model_finetuned.add(Dense(19, activation='sigmoid'))
model_finetuned.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['accuracy'])
model_finetuned.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 131072)            23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 512)               67109376  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 19)                9747      
Total params: 90,969,491
Trainable params: 70,797,331
Non-trainable params: 20,172,160
_________________________________

In [0]:
history_1 = model_finetuned.fit_generator(train_dataset, 
                                  steps_per_epoch=100, 
                                  epochs=1,
                                  validation_data=valid_dataset, 
                                  validation_steps=100, 
                                  verbose=1)

Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 551 could not be retrieved. It could be because a worker has died.
  UserWarning)


  2/100 [..............................] - ETA: 1:07:09 - loss: 0.7677 - accuracy: 0.5913

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 461 could not be retrieved. It could be because a worker has died.
  UserWarning)


  3/100 [..............................] - ETA: 1:06:19 - loss: 0.6779 - accuracy: 0.6494

...

  5/100 [>.............................] - ETA: 1:07:04 - loss: 0.5532 - accuracy: 0.7350

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 124 could not be retrieved. It could be because a worker has died.
  UserWarning)


  6/100 [>.............................] - ETA: 1:07:24 - loss: 0.5082 - accuracy: 0.7671

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 546 could not be retrieved. It could be because a worker has died.
  UserWarning)


KeyboardInterrupt: ignored

In [0]:
test_dataset = test_datagen.flow_from_directory('/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Split/train', target_size=(256, 256), batch_size=1, class_mode='categorical', shuffle = False)

Found 62083 images belonging to 19 classes.


In [0]:
test_dataset.reset()
predict = model_finetuned.predict_generator(test_dataset, steps=27, callbacks=None, workers=1, use_multiprocessing=False, verbose=1)


27/27 [==============================] - 15s 552ms/step


In [0]:
#print(predict)
predicted_class_indices=np.argmax(predict,axis=1)
labels = (test_dataset.class_indices)
labels2 = dict((v,k) for k,v in labels.items())
predictions = (labels(k) for k in predicted_class_indices)
print(predicted_class_indices)
print (labels)
#print (predictions)
labels = (test_dataset.class_indices)
labels2 = dict((v,k) for k,v in labels.items())

#from sklearn.metrics import confusion_matrix

#y_true = np.argmax(predict,axis=2)
#y_pred = predict > 0.5

#confusion_matrix(y_true, y_pred)

[13  3 15 13 13 13 13 13 13 13 13 13  3 13 13 13 13 13 15  3 15  3 15 15
 13 13 13]
{'Abstract_Expressionism': 0, 'Art_Nouveau_Modern': 1, 'Baroque': 2, 'Color_Field_Painting': 3, 'Cubism': 4, 'Early_Renaissance': 5, 'Expressionism': 6, 'High_Renaissance': 7, 'Impressionism': 8, 'Mannerism_Late_Renaissance': 9, 'Minimalism': 10, 'Naive_Art_Primitivism': 11, 'Northern_Renaissance': 12, 'Pop_Art': 13, 'Post_Impressionism': 14, 'Realism': 15, 'Rococo': 16, 'Romanticism': 17, 'Symbolism': 18}


In [0]:
model_finetuned.save('art_style.h5')

In [0]:
    plt.plot(history_1['acc'])
    plt.plot(history_1['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history_1['loss'])
    plt.plot(history_1['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

TypeError: ignored